In [9]:
from Experiment_analysis import *
import numpy as np
import pandas as pd
import random
import os

ENVIRONMENTS = []
for name, element in Environment.__members__.items():
    ENVIRONMENTS.append(name)
    
NV_DICT = Environment.__dict__

## Generating Experiment Files

In [2]:
# Generate randomly generated actions
n = 8
k = 6
scale = 1
decimals = 2
granularity = 10**decimals
all_actions = []
for env_i in range(3):
    if env_i == 2:
        scale = 2
    actions = []
    for i in range(n):
        # Generate action between 0 and 1 on with step sizes of 0.01
        _pos = [x/granularity for x in random.sample(range(scale*granularity), int(k/2))]
        _ = [_pos.append(-x/granularity) for x in random.sample(range(scale*granularity), int(k/2))]
        _acts = _pos   
        _acts.sort()
        print(_acts)
        actions.append(_acts)
    all_actions.append(actions)


[-0.4, -0.26, -0.19, 0.07, 0.13, 0.9]
[-0.86, -0.66, -0.63, 0.1, 0.56, 0.89]
[-0.59, -0.54, -0.52, 0.53, 0.59, 0.75]
[-0.88, -0.29, -0.22, 0.39, 0.79, 0.93]
[-0.82, -0.69, -0.6, 0.16, 0.42, 0.9]
[-0.22, -0.06, -0.01, 0.7, 0.82, 0.9]
[-0.55, -0.53, -0.02, 0.14, 0.47, 0.48]
[-0.83, -0.8, -0.77, 0.09, 0.91, 0.97]
[-0.79, -0.67, -0.44, 0.37, 0.48, 0.84]
[-0.68, -0.36, -0.35, 0.14, 0.47, 0.97]
[-0.99, -0.6, -0.15, 0.39, 0.66, 0.85]
[-0.76, -0.49, -0.03, 0.02, 0.31, 0.65]
[-0.9, -0.48, -0.36, 0.05, 0.6, 0.78]
[-0.82, -0.52, -0.44, 0.17, 0.19, 0.53]
[-0.8, -0.07, -0.01, 0.0, 0.42, 0.81]
[-0.72, -0.52, -0.16, 0.23, 0.29, 0.77]
[-1.6, -1.27, -0.07, 0.62, 1.16, 1.21]
[-1.07, -1.05, -0.51, 0.49, 1.38, 1.5]
[-1.71, -1.12, -0.47, 0.07, 0.83, 1.77]
[-0.37, -0.32, -0.3, 0.03, 0.24, 0.3]
[-1.95, -0.6, -0.24, 0.64, 1.33, 1.58]
[-1.67, -0.55, -0.51, 0.92, 1.42, 1.8]
[-1.95, -1.37, -0.79, 0.0, 0.75, 1.85]
[-1.82, -1.51, -0.81, 0.7, 1.09, 1.43]


In [3]:
# run = "echo 'exp2a cpc{}'\n ./target/release/broccoli --env cpc --depth 2 --num-nodes 3 --num-iters 10000  --predicate-increment 0.1 0.1 0.05 0.1 --initial-state-values -0.05 0.05 0.05 0.05 --predicate-reasoning 1 --actions {} > small_tree_cpc_s={}.txt"
# 
# with open("action_exp/2a_cpc.sh", "w") as f:
#     for i, x in enumerate(all_actions[0]):
#         string = ""
#         for y in x:
#             string += "{:.3f} ".format(y)
#         f.write(run.format(i,string, i))
#         f.write("\n")
        
    

In [4]:
# run = "echo 'exp2a mcc{}'\n ./target/release/broccoli --env mcc --depth 2 --num-nodes 3 --num-iters 1000 --predicate-increment 0.1 0.01 --initial-state-values -0.5 0 --predicate-reasoning 1 --actions {} > small_tree_mcc_s={}.txt"
# 
# 
# with open("action_exp/2a_mcc.sh", "w") as f:
#     
#     for i, x in enumerate(all_actions[1]):
#         string = ""
#         for y in x:
#             string += "{:.3f} ".format(y)
#         f.write(run.format(i,string, i))
#         f.write("\n")
        
    

In [27]:
run = "echo 'exp2a penc{}'\n ./target/release/broccoli --env penc --depth 2 --num-nodes 3 --num-iters 100 --predicate-increment 0.1 0.1 --initial-state-values 0.5 0 --predicate-reasoning 1 --actions {} > small_tree_penc_s={}.txt"

with open("action_exp/2a_penc.sh", "w") as f:

    for i, x in enumerate(all_actions[2]):
        string = ""
        for y in x:
            string += "{:.3f} ".format(y)
        f.write(run.format(i,string, i))
        f.write("\n")

#     

NameError: name 'all_actions' is not defined

## Analysis

In [10]:
import os
PATH = 'data/exp2/exp2SOT'
prune_dict = {"cpc":[], "mcc":[], "penc":[]}

for entry in os.scandir(PATH):
    if entry.is_file():
        _tmp = entry.name.split("_")[2]         # this is env
        if _tmp in ENVIRONMENTS:
            prune_dict[_tmp].append(parse_broccoli2(entry,_tmp))


In [11]:
dfs = []
for env in ENVIRONMENTS:
    dfs.append(pd.DataFrame(create_data_frame(prune_dict[env])))

print(dfs)

In [4]:
results = dfs[2]
results.head(100)

In [12]:
top = 3
indices_mcc = results.sort_values("score").index[-top:]
print(indices_mcc)
good_outputs = []
actions = []
for i in indices_mcc:
    broc = prune_dict["cpc"][i]
    good_outputs.append(broc)
    # print(f"id: {i}, score: {broc.score:.3f}")
    g_actions = get_actions(broc.tree, broc.actions)
    print(f"Used actions: {g_actions}")
    for action in g_actions.values():
        actions.append(action)
print(actions)

In [6]:
n_actions = 10
final_actions = []
for a, c in actions:
    final_actions.append(a)
        
sorted_actions = sorted(final_actions)
print(sorted_actions)

            
    

In [7]:
run_cpc = "echo 'exp2a cp final' \n./target/release/broccoli --env cpc --depth 2 --num-nodes 3 --num-iters 10000 --action-policy none --predicate-increment 0.1 0.1 0.05 0.1 --initial-state-values -0.05 0.05 0.05 0.05 --predicate-reasoning 1 --actions {} > small_trees_cpc_final.txt"
run_penc = "echo 'exp2a penc final' \n./target/release/broccoli --env penc --depth 2 --num-nodes 3 --num-iters 100 --action-policy none --predicate-increment 0.1 0.1 --initial-state-values 0.5 0 --predicate-reasoning 1 --actions {}  > small_trees_penc_final.txt"
run_mcc = "echo 'exp2a mcc final' \n./target/release/broccoli --env mcc --depth 2 --num-nodes 3 --num-iters 1000 --action-policy none --predicate-increment 0.1 0.014 --initial-state-values -0.5 0.0 --predicate-reasoning 1 --actions {} > small_trees_mcc_final.txt"

###### CHANGE THIS ######
run = run_cpc
string = ""
for y in sorted_actions:
    string += "{:.3f} ".format(y)

###### CHANGE THIS ######
with open("action_exp/2b_cpc_final.sh", "w") as f:
    f.write(run.format(string))
    f.write("\n")



# Final Analysis